OneLayerModel - 3x3 kernel

In [32]:
import torch
import torch.nn as nn
# import torchsummary
import torch.optim as optim

class OneLayerModel(nn.Module):
    def __init__(self):
        super(OneLayerModel, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=3, stride=1, padding=0, bias=False)
        
    def forward(self, x):
        x = self.conv1(x)
        return x

# only one image
# imgs = torch.tensor([[[[0, 1, 1] for _ in range(3)]], dtype=torch.float32) # [1, 1, 3, 3] = [batch, channel, h, w]
# labels = torch.tensor([[[[3]]], dtype=torch.float32)

# two images
# imgs = torch.tensor([[[[0, 1, 1] for _ in range(3)]], [[[1, 1, 0] for _ in range(3)]]], dtype=torch.float32) # [1, 1, 3, 3] = [batch, channel, h, w]
# labels = torch.tensor([[[[3]]], [[[-3]]]], dtype=torch.float32)

'''
tensor([[[[-1.1521, -0.0302,  0.8420],
          [-0.9476,  0.1549,  1.0120],
          [-0.9611, -0.0593,  1.0756]]]])
          
tensor([[[[-0.7730,  0.0423,  0.8390],
          [-1.0702,  0.3320,  1.1915],
          [-1.3074, -0.2187,  0.8084]]]]) 
'''

# three images
# imgs = torch.tensor([[[[0, 1, 1] for _ in range(3)]], [[[1, 1, 0] for _ in range(3)]], [[[1, 1, 1] for _ in range(3)]]], dtype=torch.float32) # [1, 1, 3, 3] = [batch, channel, h, w]
# labels = torch.tensor([[[[3]]], [[[-3]]], [[[0]]]], dtype=torch.float32)

'''
tensor([[[[-0.9245,  0.1166,  0.9693],
          [-1.0933, -0.1440,  1.1173],
          [-1.0377,  0.1175,  0.8462]]]])
'''

# four images
imgs = torch.tensor([[[[0, 1, 1] for _ in range(3)]], [[[1, 1, 0] for _ in range(3)]], [[[1, 1, 1] for _ in range(3)]], [[[0, 0, 0] for _ in range(3)]]], dtype=torch.float32) # [1, 1, 3, 3] = [batch, channel, h, w]
labels = torch.tensor([[[[3]]], [[[-3]]], [[[0]]], [[[0]]]], dtype=torch.float32)

'''
tensor([[[[-0.9075, -0.0911,  0.8535],
          [-1.1970, -0.0158,  1.0339],
          [-0.8708,  0.0785,  1.1265]]]]) 
'''

model = OneLayerModel()

criterion = nn.MSELoss()

optimizer = optim.SGD(model.parameters(), lr=0.01)

num_epoch = 100

def print_param():
    global model
    for param in model.parameters():
        print(param.data, '\n')
        
print_param()

for epoch in range(1, num_epoch+1):
    print(f'### Epoch_{epoch} ###\n')
    
    for img, label in zip(imgs, labels):
    
        optimizer.zero_grad()
        
        outputs = model(img)
        
        print(f'The output is {outputs}\n')
        
        loss = criterion(outputs, label)
        
        print(f'Loss is {loss}\n')
        
        optimizer.zero_grad()
        
        loss.backward()
        
        optimizer.step()
        
        print_param()

tensor([[[[ 0.1798, -0.1643, -0.3176],
          [-0.1097, -0.0890, -0.1372],
          [ 0.2165,  0.0053, -0.0446]]]]) 

### Epoch_1 ###

The output is tensor([[[-0.7472]]], grad_fn=<SqueezeBackward1>)

Loss is 14.04169750213623

tensor([[[[ 0.1798, -0.0893, -0.2426],
          [-0.1097, -0.0140, -0.0622],
          [ 0.2165,  0.0803,  0.0304]]]]) 

The output is tensor([[[0.2637]]], grad_fn=<SqueezeBackward1>)

Loss is 10.651474952697754

tensor([[[[ 0.1146, -0.1546, -0.2426],
          [-0.1749, -0.0793, -0.0622],
          [ 0.1513,  0.0150,  0.0304]]]]) 

The output is tensor([[[-0.4025]]], grad_fn=<SqueezeBackward1>)

Loss is 0.1619895100593567

tensor([[[[ 0.1226, -0.1466, -0.2346],
          [-0.1669, -0.0712, -0.0542],
          [ 0.1593,  0.0231,  0.0384]]]]) 

The output is tensor([[[0.]]], grad_fn=<SqueezeBackward1>)

Loss is 0.0

tensor([[[[ 0.1226, -0.1466, -0.2346],
          [-0.1669, -0.0712, -0.0542],
          [ 0.1593,  0.0231,  0.0384]]]]) 

### Epoch_2 ###

The ou

TwoLayerModel - 3x3 kernel - 5x5 image

In [106]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# import torchsummary

import numpy as np
import pandas as pd

import plotly.express as px

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Device is {device}\n')

class TwoLayerModel(nn.Module):
    def __init__(self):
        super(TwoLayerModel, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=3, stride=1, padding=0, bias=False)
        self.fc1 = nn.Linear(9, 1, bias=False)
        # self.fc1 = nn.Linear(9, 2, bias=False)
        self.softmax = nn.Softmax(1)
        
    def forward(self, x):
        x = x.to(torch.float32)
        x = F.relu(self.conv1(x))
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        # x = self.softmax(x)
        x = torch.sigmoid(x)
        return x

# random 100 images
imgs = []
labels = []
label_list = []

n = 512
i, j = 0, 0
while i < n or j < n:
    row = torch.randint(0, 2, (5,))
    if torch.all(row == 0) or torch.all(row == 1):
        if i < n:
            label = torch.tensor([[0]], dtype=torch.float32)
            # label = torch.tensor([0], dtype=torch.float32)
            label_ = 0
            i += 1
            # print('line x', row)
        else:
            continue
    else:
        consecutive = False
        for k in range(4):
            if row[k] == row[k+1]:
                consecutive = True
                break
        if consecutive:
            if j < n:
                label = torch.tensor([[1]], dtype=torch.float32)
                # label = torch.tensor([1], dtype=torch.float32)
                label_ = 1
                j += 1
                # print('line o', row)
            else:
                continue
        else:
            if i < n:
                label = torch.tensor([[0]], dtype=torch.float32)
                # label = torch.tensor([0], dtype=torch.float32)
                label_ = 0
                i += 1
                # print('line x', row)
            else:
                continue
        
    img = row.repeat(5, 1).unsqueeze(0) # (1,5,5)
    imgs.append(img)
    labels.append(label)
    label_list.append(label_)

model = TwoLayerModel()

model.to(device)
imgs = torch.stack(imgs).to(device)
labels = torch.stack(labels).to(device)

indices = np.arange(2*n)
np.random.shuffle(indices)
imgs, labels = imgs[indices], labels[indices]

# criterion = nn.MSELoss()
criterion = nn.BCELoss()

learning_rate = 0.01
# optimizer = optim.SGD(model.parameters(), lr=learning_rate)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

num_epoch = 32

def print_param():
    global model
    for param in model.parameters():
        print(param.data, '\n')
        
print_param()

for epoch in range(1, num_epoch+1):
    print(f'### Epoch_{epoch} ###\n')
    
    loss_sum = 0
    output_list = []
    label_list = []
    
    for img, label in zip(imgs, labels):
    
        optimizer.zero_grad()
        
        output = model(img)
        
        loss = criterion(output, label)
        
        optimizer.zero_grad()
        
        loss.backward()
        
        optimizer.step()
        
        # print(f'The output is {output}\n')
        
        # print(f'Loss is {loss}\n')
        
        # print_param()
        
        loss_sum += loss
        output_list.append(round(output.item(), 2))
        label_list.append(label.item())
    
    loss_sum = round(loss_sum.item() / n, 3)
        
    print(f'Epoch loss is {loss_sum}\n')
    print(f'Outputs: {output_list[0:10]}')
    print(f'Labels:  {label_list[0:10]}\n')

print_param()

p_list = []
r_list = []
f1_list = []

threshold_list = [i / 100 for i in range(101)] # 0, 0.01, ..., 1.00
for threshold in threshold_list:

    output_th= [1.0 if x > threshold else 0.0 for x in output_list]

    tp, tn, fp, fn = 0, 0, 0, 0
    for o, l in zip(output_th, label_list):
        if o == l:
            if o == 1.0:
                tp += 1
            else:
                tn += 1
        else:
            if o == 1.0:
                fp += 1
            else:
                fn += 1
    precision = tp / (tp+fp+0.001)
    recall = tp / (tp+fn+0.001)
    f1_score = 2 * (precision * recall) / (precision + recall+ 0.001)
    
    p_list.append(precision)
    r_list.append(recall)
    f1_list.append(f1_score)
    
    
df = pd.DataFrame({'T': threshold_list, 'P': p_list, 'R': r_list, 'F1': f1_list})

fig = px.line(df, x='T', y=['P', 'R', 'F1'], title='Precision, Recall & F1 score')
fig.show()

Device is cuda

tensor([[[[ 0.0278,  0.2676, -0.0180],
          [ 0.2433, -0.1290,  0.0382],
          [ 0.0668,  0.2975,  0.0502]]]], device='cuda:0') 

tensor([[-0.0600, -0.1601,  0.0104, -0.1591,  0.0815,  0.2578,  0.3255,  0.1186,
         -0.2159]], device='cuda:0') 

### Epoch_1 ###

Epoch loss is 1.091

Outputs: [0.52, 0.5, 0.5, 0.5, 0.59, 0.57, 0.63, 0.61, 0.61, 0.57]
Labels: [1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0]

### Epoch_2 ###

Epoch loss is 0.896

Outputs: [1.0, 0.5, 0.5, 0.5, 0.99, 0.5, 0.99, 0.5, 1.0, 0.5]
Labels: [1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0]

### Epoch_3 ###

Epoch loss is 0.895

Outputs: [1.0, 0.5, 0.5, 0.5, 1.0, 0.5, 1.0, 0.5, 1.0, 0.5]
Labels: [1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0]

### Epoch_4 ###

Epoch loss is 0.894

Outputs: [1.0, 0.5, 0.5, 0.5, 1.0, 0.5, 1.0, 0.5, 1.0, 0.5]
Labels: [1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0]

### Epoch_5 ###

Epoch loss is 0.894

Outputs: [1.0, 0.5, 0.5, 0.5, 1.0, 0.5, 1.0, 0

KeyboardInterrupt: 